In [1]:
import pandas as pd
import os
import sys
import json
import numpy as np
import tensorflow as tf
import keras.layers as L
import keras.activations as A
import keras.optimizers as O
import keras.losses as Loss
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import backend as KB
from keras.callbacks import Callback
import time

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ROOTDIR=os.path.abspath("../../")
sys.path.append(ROOTDIR)

In [3]:
from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_generator import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back
from utils.tools import construct_model_from_csv
from models.BaseModel import Base_Model

In [4]:
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
data_dir=os.path.join(ROOTDIR,"Data")

In [5]:
files=os.listdir(data_dir)
array=pd.read_hdf(os.path.join(data_dir,files[0]))

P=preprocessor(label_encoder_dir)
array=P.process_batch(array)
action_col=array.columns=='action'

p_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':5,
        'format_':'h5',
        'ex_batch_size':10,
        'file_examples':5000,
        'max_queue_size':10,
        'Preprocessor':P,
        'static_weightage':False,
         'action_col':action_col}

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
class FC_CONV_Generator(Generator):
    def __init__(self,config):
        super().__init__(config)
    
    def __getitem__(self, idx):
        x,targets=super().__getitem__(idx)
        ob_net=(x[:,2:467]).reshape(-1,155,3,1)
#         ob_net=np.expand_dims(ob_net,axis=-1)
        num_cols=x.shape[1]
#         print (num_cols)
        scalar_features_col_mask=np.arange(num_cols)
        scalar_features_col_mask=(scalar_features_col_mask<2 ) |(scalar_features_col_mask>=467 )
        scalar_features=x[:,scalar_features_col_mask]
        
        return [ob_net,scalar_features],targets

In [7]:
G=FC_CONV_Generator(config=p_config)

In [8]:
from multiprocessing import Process
p1 = Process(target=G.generate)
p1.start()

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
# G[0][0]

In [10]:
class FC_CONV(Base_Model):
    
    def __init__(self,input_shape,num_outputs,optimizer,arch_config_files):
        """
        params:
            input_shape: a list that specifies input_shapes 
                            of all the inputs to the model (batch dim not included)
            num_outputs:num of neurons in the last layer
            optimizer: optimizer function used during training
            arch_config_files:list of 3 files that specify the design of conv network,fc network
                                and the network that combines output of both these networks
        """
        assert len(input_shape)==2
        Base_Model.__init__(self,input_shape,num_outputs,optimizer,"FC_CONV")
        assert len(arch_config_files)==3
        self.arch_config_files=arch_config_files
        
    def construct_model(self):
        """
        Model is constructed here and self.Model stores the link to the Model
        """
        conv_input=L.Input(shape=(self.input_shape[0]))
        scalar_features=L.Input(shape=(self.input_shape[1]))
        
        flattened_conv_features=construct_model_from_csv(self.arch_config_files[0],conv_input)
        fc_features=construct_model_from_csv(self.arch_config_files[1],scalar_features)
        concatenated_features=L.concatenate([flattened_conv_features,fc_features])
        logits=construct_model_from_csv(self.arch_config_files[2],concatenated_features)
        self.Model=Model(inputs=[conv_input,scalar_features],output=logits)
        
#         f1=f1_score_metric(self.output)        
        self.Model.compile(loss=weighted_cross_entropy_loss, optimizer=self.optimizer,metrics=["acc"])
        
        self.Model.summary()
        
        
        

In [11]:
input_shape=[(155,3,1),(8,)]
num_outputs=p_config['labels']
optimizer=O.Adam(lr=0.0001)
design_config_files=["conv_net.csv","fc_net.csv","merged_net.csv"]
model=FC_CONV(input_shape=input_shape,num_outputs=num_outputs,
        optimizer=optimizer,arch_config_files=design_config_files)
model.construct_model()

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 155, 3, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 155, 3, 128)  1280        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 155, 3, 128)  512         conv2d_1[0][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [12]:
model.train(G)

Epoch 1/10


ValueError: Error when checking input: expected input_2 to have shape (8,) but got array with shape (7,)

In [ ]:
# n_config={'Hidden':[200],
#           'Activation':'relu',
#           'Output':p_config['labels'],
#           'Optimizer':O.Adam(lr=0.0001),
#           'Input_shape':477,'epochs':1 ,
#           'loss':weighted_cross_entropy_loss}
# model=FC_Model(n_config)

In [ ]:
# model.fit(generator=G)

In [ ]:
# # # a={"layer_type":["abc"],"units":[None],"strides":[""],"padding":[""],"kernel":[""]}
# # a={"layer_type": [""], "units":[""],"strides":[""],
# #                                    "padding":[""],"kernel_size":[""],"num_filters":[""]}
# # # columns=["layer_type","units","strides","padding","kernel_size","num_filters"]
# # df=pd.DataFrame(a)
# # df.to_csv("model.csv")
# df=pd.read_csv("model.csv")
# print (df)